# Toy Problem Demo

## Scope

What is a pipeline?

How to create a pipeline in benchq:
- inputs and outputs
- components:
  - Transpilation (pyliqtr)
  - Jabalizer/ICM
  - Min code distance finding
  - Substrate scheduling

#### Goal: Introduction to Benchq's Parts

## Agenda
- Some example pipelines
- Quick overview of Benchq usage.
- What is a circuit graph? How do we produce them?
- How do circuit graphs help get resource estimates?
- Look at some pretty plots
  - Preparing a GHZ state
  - The fully connected graph

In [ ]:
%matplotlib inline

## Some Example Pipelines

### The Simplest Pipeline

The simplest pipeline is the automatic pipeline

To use it, you'll need:
- 2 Input Objects
  - AlgorithmDescription
  - Hardware Model

- 1 Output Object
  - ResourceEstimation

In [ ]:
from benchq import automatic_resource_estimator, SCArchitectureModel, get_algorithm_description_from_circuit, ErrorBudget
from qiskit import QuantumCircuit

demo_circuit = QuantumCircuit.from_qasm_file("circuits/single_rotation.qasm")

error_budget = ErrorBudget(1e-3) # default error budget
algorithm_description = get_algorithm_description_from_circuit(demo_circuit, error_budget)

automatic_resource_estimator(algorithm_description, SCArchitectureModel)

Getting resource estimates is easy!

### What's in an `AlgorithmDescription`?

2 Parts:
- A `QuantumProgram`
- An `ErrorBudget`

#### Quantum Programs

Your program is a description of the circuit you want to perform.

In this case, we can just use the circuit itself. But for larger circuits we might want to break them into peices. (more on this later)

In [ ]:
from benchq.data_structures import get_program_from_circuit

demo_circuit = QuantumCircuit.from_qasm_file("circuits/rotation_cnot.qasm")
demo_program = get_program_from_circuit(demo_circuit)

#### Error Budget

Here we describe how each of the error sources will contribute to the overall error.

We'll just use the default error budget for this tutorial.

In [ ]:
from benchq.data_structures import ErrorBudget

ultimate_failure_tolerance = 0.01  # obtained from TA1 teams
error_budget = ErrorBudget(ultimate_failure_tolerance)

### What are the different Peices of the Resource Estimator?

For clarity, we will follow one particular pipeline:

`run_estimate_using_graph_estimator_without_delayed_gate_synthesis`

there are others, but they are out of the scope of this presentation.

It can be called the same as `automatic_resource_estimator`.

In [ ]:
from benchq.resource_estimation.graph import run_estimate_using_graph_estimator_without_delayed_gate_synthesis

run_estimate_using_graph_estimator_without_delayed_gate_synthesis(demo_program, error_budget)

But what does this function actually do?

1. Transpiles the circuit to clifford + T
2. Gets the graph corresponding to that circuit
3. Gets estimates from that graph.

In [ ]:
from benchq.resource_estimation.graph import synthesize_clifford_t

transformer = synthesize_clifford_t(error_budget)

demo_program =transformer(demo_program, circuit_precision=1e-6)
print(demo_program.full_circuit)

Transform circuit into graph.

In [ ]:
from benchq.resource_estimation.graph import create_big_graph_from_subcircuits

transformer = create_big_graph_from_subcircuits()

demo_program = transformer(demo_program)

With this use this graph to make resource estimates.

In [ ]:
from benchq.resource_estimates.graph import GraphResourceEstimator

estimator = GraphResourceEstimator(SCArchitectureModel())

resource_estimates = estimator.get_resource_estimations_for_graph(demo_program, architecture_model)
print(resource_estimates)

### Summary

#### Inputs:
- Algorithm description
  - description of your circuit (QuantumProgram)
  - error tolerances (ErrorBudget)
- Archetecture Model

#### Outputs:
- Number of physical qubits
- Computation time
- number of measurement steps (will be important later on!)

## What is a a Circuit Graph?

In [ ]:
circuit_graph = get_algorithmic_graph_from_Jabalizer(demo_program.full_circuit)

### What does this do?

Recall that our circuit is in clifford + T form

- Replaces T gates with magic measurements and ancilla
- Use stabilizer simulator efficiently to push single qubit cliffords to one side
- Now we have a circuit of the form Initialization, CNOT, Measurement (ICM form)

In [ ]:
circuit_before_icm = json.load(open("data/icm_input_circuit.json"))
print(circuit_before_icm)

In [ ]:
circuit_after_icm = json.load(open("data/icm_output.json"))
print(circuit_after_icm)

The middle CNOTS are the interesting part:

- The CNOTS make a graph state
- Use stabilizer simulator to find graph state (Jabalizer)
- Return graph state as circuit graph

In [ ]:
nx.draw(circuit_graph, node_size=10)

### Summary

Circuit graphs are a simplify circuits.

Count T-gate resources separately.

## Getting Resource Estimates from Circuit Graphs

circuit graph state + measurement = circuit implementation

At the physical level, how many qubits do we need?

At the logical level, how do we make the graph state?

### How many qubits do we need?

`find_min_viable_distance` tries a bunch of different code distances. (the power of the code)

Returns the number of physical qubits required to reach that distance.

In [ ]:
from benchq.resource_estimation.graph_compilation import find_min_viable_distance

logical_qubit_count = len(circuit_graph)
distance = find_min_viable_distance(
    logical_qubit_count,
    architecture_model.physical_gate_error_rate, # physical error rate
    10e-3, # logical error rate
)

physical_qubit_count = 12 * logical_qubit_count * 2 * distance**2
total_time = 240 * logical_qubit_count * distance * 6 * architecture_model.physical_gate_time_in_seconds


print(f"distance: {distance}")
print(f"physical qubit count: {physical_qubit_count}")
print(f"total time: {total_time}")

### How to make Circuit Graph State?

Since graph state is a stabilizer state, we measure stabilizers to generate it!

We could measure all the stabilizers to get the graph.

Measurements are expensive!! So how optimize?

### Substrate Scheduler

Tells us how to measure and which can be measured simultaneously.

In [ ]:
from benchq.resource_estimation.graph_compilation import substrate_scheduler

compiler = substrate_scheduler(circuit_graph)
formatted_measurement_steps = [[node[0] for node in step] for step in compiler.measurement_steps]
print(formatted_measurement_steps)

In [ ]:
from benchq.vizualization_tools import plot_graph_state_with_measurement_steps

plot_graph_state_with_measurement_steps(compiler.input_graph, compiler.measurement_steps)

### Problem! Graph can get too big to handle!

#### Solution! Use subcircuits.

Quantum Algorithms are made up of repeated components.

Estimate resources for each component & multiply by the number of times it was used.

Will create a higher estimate.

More on this later!

## FINALLY! Pretty Graph Time!

Let's look at the graphs of circuits to examine measurement steps!

In [ ]:
circuit = QuantumCircuit.from_qasm_file("circuits/ghz_circuit.qasm")

clifford_t_circuit = pyliqtr_transpile_to_clifford_t(circuit, circuit_precision=1e-10)
circuit_graph = get_algorithmic_graph_from_Jabalizer(clifford_t_circuit)
ghz_resource_estimates = get_resource_estimations_for_graph(circuit_graph, architecture_model, 1e-3, plot=True)
print(ghz_resource_estimates)

In [ ]:
circuit = QuantumCircuit.from_qasm_file("circuits/h_chain_circuit.qasm")

clifford_t_circuit = pyliqtr_transpile_to_clifford_t(circuit, circuit_precision=1e-10)
circuit_graph = get_algorithmic_graph_from_Jabalizer(clifford_t_circuit)
h_chain_resource_estimates = get_resource_estimations_for_graph(circuit_graph, architecture_model, 1e-3, plot=True)
print(h_chain_resource_estimates)

## Closing Statements

### What did we learn?


#### Inputs
- Circuit
- Architecture model
#### Outputs
- Number of physical qubits
- Computation time
- Number of measurement steps



#### Components:
- Transpilation (pyliqtr)
  - Bring to Clifford + T
- Jabalizer/ICM
  - Easy way to represent circuit
- Min code distance finding
  - Number of physical qubits
  - Computation time
- Substrate scheduling
  - number of measurement steps

## What's Next?

- How to get resource estimate for large algorithms? (QuantumPrograms)
- Compare to other resource estimators.
- Try this notebook out for yourself!!